In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**1. Data Processing:**

a) Import the data: You are provided separate .csv files for train and test.


In [0]:
filepath = '/content/gdrive/My Drive/Machine Learning/Assignment 5/Urban land cover/'
train_path = filepath + 'train_data.csv'
test_path = filepath + 'test_data.csv'

In [0]:
import pandas as pd
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [4]:
train.head(5)

,class,BrdIndx,Area,Round,Bright,Compact,ShpIndx,Mean_G,Mean_R,Mean_NIR,...,SD_NIR_140,LW_140,GLCM1_140,Rect_140,GLCM2_140,Dens_140,Assym_140,NDVI_140,BordLngth_140,GLCM3_140
0,concrete,1.32,131,0.81,222.74,1.66,2.18,192.94,235.11,240.15,...,31.15,5.04,0.80,0.58,8.56,0.82,0.98,-0.10,1512,1287.52
1,shadow,1.59,864,0.94,47.56,1.41,1.87,36.82,48.78,57.09,...,12.01,3.70,0.52,0.96,7.01,1.69,0.86,-0.14,196,2659.74
2,shadow,1.41,409,1.00,51.38,1.37,1.53,41.72,51.96,60.48,...,18.75,3.09,0.90,0.63,8.32,1.38,0.84,0.10,1198,720.38
3,tree,2.58,187,1.91,70.08,3.41,3.11,93.13,55.20,61.92,...,27.67,6.33,0.89,0.70,8.56,1.10,0.96,0.20,524,891.36
4,asphalt,2.60,116,2.05,89.57,3.06,3.02,73.17,94.89,100.64,...,32.05,1.01,0.83,0.75,8.62,2.08,0.08,-0.10,496,1194.76


In [0]:
# b) Remove any rows that have missing data across both sets of data
train = train.dropna(how='any', axis = 0)
test = test.dropna(how='any', axis = 0)

In [0]:
#c) The target variable (dependent variable) is called "class", make sure to separate this out into a "y_train" and "y_test" and remove from your "X_train" and "X_test"
y_train = train['class']
y_test = test['class']
X_train = train.drop(columns='class')
X_test = test.drop(columns='class')

In [0]:
# d) Scale all features / predictors (NOT THE TARGET VARIABLE)
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train))
X_test_scaled = pd.DataFrame(scaler.transform(X_test))

In [0]:
#update column names of the X dataframes
X_train_scaled.columns = X_train.columns.values
X_test_scaled.columns = X_test.columns.values

**2. Random Forest Classifier - Base Model:**

Start by creating a simple Random Forest only using default parameters - this will let us compare SVMs to Random Forest in multiclass problems.

a) Use the RandomForestClassifier in sklearn. Fit your model on the training data.



In [0]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
base_model = RFC.fit(X_train_scaled, y_train)

b) Use the fitted model to predict on test data. Use the .predict() method to get the predicted classes.

In [0]:
y_pred_base = base_model.predict(X_test_scaled)

c) Calculate the confusion matrix and classification report (both are in sklearn.metrics)

In [11]:
#Calculate and print confusion matrix
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test, y_pred_base)
print("Confusion Matrix:")
print(cnf_matrix)

Confusion Matrix:
[[14  0  0  0  0  0  0  0  0]
 [ 1 20  1  3  0  0  0  0  0]
 [ 0  3 11  1  0  0  0  0  0]
 [ 0  4  0 19  0  0  0  0  0]
 [ 0  0  0  0 23  0  0  0  6]
 [ 1  0  1  0  0 13  0  0  0]
 [ 2  0  0  0  0  0 14  0  0]
 [ 0  2  0  5  3  0  0  4  0]
 [ 0  0  0  1  2  0  0  0 14]]


In [12]:
# Calculate and print classification report

from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(y_test, y_pred_base))

Classification Report:
             precision    recall  f1-score   support

   asphalt        0.78      1.00      0.88        14
  building        0.69      0.80      0.74        25
       car        0.85      0.73      0.79        15
  concrete        0.66      0.83      0.73        23
     grass        0.82      0.79      0.81        29
      pool        1.00      0.87      0.93        15
    shadow        1.00      0.88      0.93        16
      soil        1.00      0.29      0.44        14
      tree        0.70      0.82      0.76        17

avg / total       0.81      0.79      0.78       168



d) Identify the top 5 features. Feel free to print a list OR to make a plot. 



In [13]:
import pandas as pd
feature_imp = pd.DataFrame(base_model.feature_importances_) 
feature_imp.set_index(X_train.columns.values)
feature_imp = feature_imp.rename(index=lambda x: X_train.columns[x])
feature_imp.columns = ['Feature Importance']
feature_imp.sort_values('Feature Importance',ascending=False).head(5)


,Feature Importance
Mean_R_40,0.069445
Mean_R_80,0.046583
Bright,0.043644
NDVI,0.042169
Mean_NIR_40,0.040844


The base model does a pretty good job in general of predicting the class, but as we can see some classes such as  building are performing much lower than the others. We will see if another model is able to improve this

**3. LinearSVM Classifier - Base Model:**

Create a simple LinearSVC Classifier only using default parameters.



In [14]:
#a) Use the LinearSVC in sklearn. Fit your model on the training data.
from sklearn.svm import LinearSVC
LSVC = LinearSVC()
LSVC_model = LSVC.fit(X_train_scaled, y_train)

#b) Use the fitted model to predict on test data. Use the .predict() method to get the predicted classes.
y_pred_SVC = LSVC_model.predict(X_test_scaled)

#c) Calculate the confusion matrix and classification report (both are in sklearn.metrics).
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_SVC))
print("Classification Report:")
print(classification_report(y_test, y_pred_SVC))

Confusion Matrix:
[[13  0  0  0  0  0  1  0  0]
 [ 0 22  1  1  1  0  0  0  0]
 [ 0  2 12  0  0  0  0  0  1]
 [ 1  6  0 15  0  0  0  0  1]
 [ 0  0  0  1 26  0  0  0  2]
 [ 1  0  1  0  0 13  0  0  0]
 [ 2  0  0  0  0  0 14  0  0]
 [ 0  4  0  1  3  0  0  6  0]
 [ 0  0  0  1  6  0  0  0 10]]
Classification Report:
             precision    recall  f1-score   support

   asphalt        0.76      0.93      0.84        14
  building        0.65      0.88      0.75        25
       car        0.86      0.80      0.83        15
  concrete        0.79      0.65      0.71        23
     grass        0.72      0.90      0.80        29
      pool        1.00      0.87      0.93        15
    shadow        0.93      0.88      0.90        16
      soil        1.00      0.43      0.60        14
      tree        0.71      0.59      0.65        17

avg / total       0.80      0.78      0.77       168



**4. Support Vector Machine Classifier + Linear Kernel + Grid Search:**

We will now use GridSearchCV to try various hyperparameters in a SVM with linear kernel.

a) Use SVC from sklearn with kernel = "linear". Run the GridSearchCV using the following (SVMs run much faster than RandomForest):

In [0]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import numpy as np
SVC = svm.SVC(kernel = 'linear')
param_grid = {'C': np.arange(0.01,10.0,0.2) }
GSVC = GridSearchCV(SVC, param_grid, cv=5)
GSVC_fit = GSVC.fit(X_train_scaled, y_train)

b) Identify the best performing model

In [16]:
#This method outputs to best performing parameters
GSVC_fit.best_params_

{'C': 0.01}

In [17]:
#This method outputs the best performing model, and can be used for predicting on the X_test
GSVC_fit.best_estimator_

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

c) Use the best estimator model to predict on test data. Use the .predict() method to get the predicted classes.

In [0]:
GSVC_pred = GSVC_fit.best_estimator_.predict(X_test_scaled)

d) Calculate the confusion matrix and classification report (both are in sklearn.metrics)

In [19]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, GSVC_pred))
print("Classification Report:")
print(classification_report(y_test, GSVC_pred))

Confusion Matrix:
[[13  0  0  0  0  0  1  0  0]
 [ 0 22  0  2  1  0  0  0  0]
 [ 0  1 14  0  0  0  0  0  0]
 [ 0  5  0 17  0  0  0  1  0]
 [ 0  0  0  1 25  0  0  0  3]
 [ 0  0  0  0  0 14  1  0  0]
 [ 1  0  0  0  0  0 15  0  0]
 [ 0  3  0  5  2  0  0  4  0]
 [ 0  0  0  1  2  0  0  0 14]]
Classification Report:
             precision    recall  f1-score   support

   asphalt        0.93      0.93      0.93        14
  building        0.71      0.88      0.79        25
       car        1.00      0.93      0.97        15
  concrete        0.65      0.74      0.69        23
     grass        0.83      0.86      0.85        29
      pool        1.00      0.93      0.97        15
    shadow        0.88      0.94      0.91        16
      soil        0.80      0.29      0.42        14
      tree        0.82      0.82      0.82        17

avg / total       0.83      0.82      0.81       168



**5. Support Vector Machine Classifier + Polynomial Kernel + Grid Search:**

We will now use GridSearchCV to try various hyperparameters in a SVM with a polynomial kernel.

a) Use SVC from sklearn with kernel = "poly". Run the GridSearchCV using the following:

C: 0.01 - 10 in increments of 0.2

degree: 2, 3, 4, 5, 6

In [0]:
SVC = svm.SVC(kernel = 'linear')
param_grid = {'C': np.arange(0.01,10.0,0.2) ,'degree':[2,3,4,5,6]}
GSVC_poly = GridSearchCV(SVC, param_grid, cv=5)
GSVC_poly_fit = GSVC.fit(X_train_scaled, y_train)

b) Identify the best performing model:

In [21]:
#This method outputs to best performing parameters
print(GSVC_poly_fit.best_params_)
#This method outputs the best performing model, and can be used for predicting on the X_test
print(GSVC_poly_fit.best_estimator_)

{'C': 0.01}
SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


c) Use the best estimator model to predict on test data. Use the .predict() method to get the predicted classes.

In [0]:
GSVC_poly_pred = GSVC_fit.best_estimator_.predict(X_test_scaled)

d) Calculate the confusion matrix and classification report (both are in sklearn.metrics)

In [23]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, GSVC_poly_pred))
print("Classification Report:")
print(classification_report(y_test, GSVC_poly_pred))

Confusion Matrix:
[[13  0  0  0  0  0  1  0  0]
 [ 0 22  0  2  1  0  0  0  0]
 [ 0  1 14  0  0  0  0  0  0]
 [ 0  5  0 17  0  0  0  1  0]
 [ 0  0  0  1 25  0  0  0  3]
 [ 0  0  0  0  0 14  1  0  0]
 [ 1  0  0  0  0  0 15  0  0]
 [ 0  3  0  5  2  0  0  4  0]
 [ 0  0  0  1  2  0  0  0 14]]
Classification Report:
             precision    recall  f1-score   support

   asphalt        0.93      0.93      0.93        14
  building        0.71      0.88      0.79        25
       car        1.00      0.93      0.97        15
  concrete        0.65      0.74      0.69        23
     grass        0.83      0.86      0.85        29
      pool        1.00      0.93      0.97        15
    shadow        0.88      0.94      0.91        16
      soil        0.80      0.29      0.42        14
      tree        0.82      0.82      0.82        17

avg / total       0.83      0.82      0.81       168



**6. Support Vector Machine Classifier + RBF Kernel + Grid Search:**

We will now use GridSearchCV to try various hyperparameters in a SVM with a RBF kernel.

a) Use SVC from sklearn with kernel = "rbf". Run the GridSearchCV using the following:

C: 0.01 - 10 in increments of 0.2
gamma: 0.1, 1, 10, 100

In [0]:
SVC = svm.SVC(kernel = 'rbf')
param_grid = {'C': np.arange(0.01,10.0,0.2) ,'gamma':[0.001,0.01,0.1,1,10,100]}
GSVC_poly = GridSearchCV(SVC, param_grid, cv=5)
GSVC_poly_fit = GSVC.fit(X_train_scaled, y_train)

b) Identify the best performing model:

In [25]:
#This method outputs to best performing parameters
print(GSVC_poly_fit.best_params_)
#This method outputs the best performing model, and can be used for predicting on the X_test
print(GSVC_poly_fit.best_estimator_)

{'C': 0.01}
SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [26]:
#c) Use the best estimator model to predict on test data. Use the .predict() method to get the predicted classes.
GSVC_rbf_pred = GSVC_fit.best_estimator_.predict(X_test_scaled)

#d) Calculate the confusion matrix and classification report (both are in sklearn.metrics)
print("Confusion Matrix:")
print(confusion_matrix(y_test, GSVC_poly_pred))
print("Classification Report:")
print(classification_report(y_test, GSVC_poly_pred))

Confusion Matrix:
[[13  0  0  0  0  0  1  0  0]
 [ 0 22  0  2  1  0  0  0  0]
 [ 0  1 14  0  0  0  0  0  0]
 [ 0  5  0 17  0  0  0  1  0]
 [ 0  0  0  1 25  0  0  0  3]
 [ 0  0  0  0  0 14  1  0  0]
 [ 1  0  0  0  0  0 15  0  0]
 [ 0  3  0  5  2  0  0  4  0]
 [ 0  0  0  1  2  0  0  0 14]]
Classification Report:
             precision    recall  f1-score   support

   asphalt        0.93      0.93      0.93        14
  building        0.71      0.88      0.79        25
       car        1.00      0.93      0.97        15
  concrete        0.65      0.74      0.69        23
     grass        0.83      0.86      0.85        29
      pool        1.00      0.93      0.97        15
    shadow        0.88      0.94      0.91        16
      soil        0.80      0.29      0.42        14
      tree        0.82      0.82      0.82        17

avg / total       0.83      0.82      0.81       168



**7. From the models run in steps 2-6, which performs the best based on the Classification Report? Support your reasoning with evidence around your test data. **

Th best model is model 4 (Support Vector Machine Classifier + Linear Kernel + Grid Search)

I chose this model because it has a higher average precision and recall percentage than the base models. Models 4-6 have the same overall precision and recall which is why I chose model 4 because it has the simplest kernel and has less of a chance of overfitting the data.

**8. Compare models run for steps 4-6 where different kernels were used. What is the benefit of using a polynomial or rbf kernel over a linear kernel? What could be a downside of using a polynomial or rbf kernel? **

Kernels allow us to enhance the feature space using a transformation 

One downside of using a polynomial or rbf kernel is that it increases the chance of overfitting the data

**9. Explain the 'C' parameter used in steps 4-6. What does a small C mean versus a large C in sklearn? Why is it important to use the 'C' parameter when fitting a model? **

The C parameter determines the sensitivity of the model to misclassification. A smaller C value will represent a less strict model that allows for some misclassification, but using a larger C will cause tight margins and may result in the model to overfitting the data.

It is important to use the C parameter to help produce a generalizable model that will not overfit the train data

**10. It is also important to check for overfitting: For your best performing model provide metrics for the training and test sets and explain whether your model is overfitting the data. **

In [28]:
GSVC_pred_train = GSVC_fit.best_estimator_.predict(X_train_scaled)
print("Test:",classification_report(y_test, GSVC_pred),"Train:",classification_report(y_train, GSVC_pred_train))

Test:              precision    recall  f1-score   support

   asphalt        0.93      0.93      0.93        14
  building        0.71      0.88      0.79        25
       car        1.00      0.93      0.97        15
  concrete        0.65      0.74      0.69        23
     grass        0.83      0.86      0.85        29
      pool        1.00      0.93      0.97        15
    shadow        0.88      0.94      0.91        16
      soil        0.80      0.29      0.42        14
      tree        0.82      0.82      0.82        17

avg / total       0.83      0.82      0.81       168
 Train:              precision    recall  f1-score   support

   asphalt        0.93      0.89      0.91        45
  building        0.85      0.90      0.87        97
       car        1.00      0.90      0.95        21
  concrete        0.87      0.89      0.88        93
     grass        0.91      0.84      0.88        83
      pool        1.00      0.86      0.92        14
    shadow        0.86      0

As we can see the precision and the recall percentages do not vary significantly from the train to the test data. This implies the model generalizable and is not overfitting the test data.